<a href="https://colab.research.google.com/github/BabyYawLwi/Flower-Detection-TFLite/blob/master/Flower_Detection_TFLite(apk)(ref).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import os


In [3]:
_URL = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'
zip_file = tf.keras.utils.get_file(origin=_URL, fname='flower_photos.tgz', extract=True, cache_subdir='/content')
base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')

228818944/228813984 [==============================] - 3s 0us/step


In [8]:
IMAGE_SIZE = 224
BATCH_SIZE = 64
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

In [4]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2
)

train_datagen = datagen.flow_from_directory(base_dir, 
                                            target_size=(IMAGE_SIZE, IMAGE_SIZE), 
                                            batch_size=BATCH_SIZE, 
                                            subset='training')
val_datagen = datagen.flow_from_directory(base_dir,
                                          target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                          batch_size=BATCH_SIZE,
                                          subset='validation')


Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [5]:
print(train_datagen.class_indices)

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [7]:
labels= '\n'.join(train_datagen.class_indices.keys())
with open('labels.txt','w') as writer:
  writer.write(labels)

In [9]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [10]:
base_model.trainable = False

In [12]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
epochs=10

history = model.fit(train_datagen, 
                    epochs=epochs,
                    validation_data = val_datagen)

Epoch 1/10
46/46 [==============================] - 127s 3s/step - loss: 1.2252 - accuracy: 0.4889 - val_loss: 0.9151 - val_accuracy: 0.5458
Epoch 2/10
46/46 [==============================] - 124s 3s/step - loss: 0.5731 - accuracy: 0.7816 - val_loss: 0.5795 - val_accuracy: 0.8140
Epoch 3/10
46/46 [==============================] - 125s 3s/step - loss: 0.2971 - accuracy: 0.8972 - val_loss: 0.4758 - val_accuracy: 0.8440
Epoch 4/10
46/46 [==============================] - 125s 3s/step - loss: 0.1923 - accuracy: 0.9411 - val_loss: 0.4651 - val_accuracy: 0.8440
Epoch 5/10
46/46 [==============================] - 128s 3s/step - loss: 0.1385 - accuracy: 0.9568 - val_loss: 0.4853 - val_accuracy: 0.8509
Epoch 6/10
46/46 [==============================] - 125s 3s/step - loss: 0.1014 - accuracy: 0.9718 - val_loss: 0.4516 - val_accuracy: 0.8605
Epoch 7/10
46/46 [==============================] - 126s 3s/step - loss: 0.0709 - accuracy: 0.9840 - val_loss: 0.4769 - val_accuracy: 0.8605
Epoch 8/10
46

In [15]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('flower_model.tflite','wb') as f:
  f.write(tflite_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: assets


In [17]:
from google.colab import files
files.download('flower_model.tflite')
files.download('labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>